This notebook removes short contiguous sequences from the *Amblyomma americanum* genome. It operates on the output of purge_dups.

<H1>Imports</H1>

In [15]:
import time

<H1>Variables</H1>

In [16]:
start = time.time()

input_file1 = 'Arcadia_Amblyomma_americanum_asm001_purged.fasta' # https://zenodo.org/record/7747102/files/Arcadia_Amblyomma_americanum_asm001_purged.fasta?download=1
output_file1 = 'Arcadia_Amblyomma_americanum_asm001_purged_cleanedup1.fasta' # https://zenodo.org/record/7783368/files/Arcadia_Amblyomma_americanum_asm001_purged_cleanedup1.fasta?download=1

#-----number of nucleotides that define short contigs which need removal (NCBI: 200nt)
short_contig_cutoff = 200

#-----list contigs needing removal
short_contigs = ['contig_9690', 'contig_6708', 'contig_93095', 'contig_5737', 'contig_125013', 'contig_104', 'contig_142567', 'contig_4598', 'contig_7758']
duplicated_contigs = ['contig_102103', 'contig_105162', 'contig_108022', 'contig_181458', 'contig_208362', 'contig_37540', 'contig_74390']
mitochondrial_contigs = ['contig_121530']

#-----dictionary (key=contig; value=2-item list containing start and stop regions) of contigs that need editing
contigs_with_regions_to_removal = {'contig_174002' : [67598,67642]}

end = time.time()
print(end-start)

0.0010139942169189453


<H1>Read in fasta file</H1>

In [17]:
start = time.time()

#-----read in genome fasta
with open(input_file1, 'r') as f:
    file1 = f.readlines()

end = time.time()
print(end-start)

80.36903810501099


<H1>Collect contigs and segments to remove</H1>

In [18]:
start = time.time()

#-----concatenate list of contigs needing removal
list_of_contigs_to_remove = short_contigs + duplicated_contigs + mitochondrial_contigs

#-----create dictionary of contigs and regions requiring editing/removal
dict_of_contigs_to_edit = contigs_with_regions_to_removal

end = time.time()
print(end-start)

0.00024008750915527344


<H1>Iterate through fasta file and output into new fasta file while filtering</H1>

In [19]:
start = time.time()

#-----define variables
entry_name1 = ''
entry_seq1 = ''

#-----iterate through read file and output fasta documents into new file
with open(output_file1, 'w') as g:
    for line1 in file1:
        if (line1[0] == '>'):
            if (entry_name1 != ''):
                #---remove whole contigs according to entries in "list_of_contigs_to_remove"
                if entry_name1 not in list_of_contigs_to_remove:
                    #---remove contig segments according to entries in "dict_of_contigs_to_edit"
                    if entry_name1 in dict_of_contigs_to_edit.keys():
                        temp_seq1 = entry_seq1[0:dict_of_contigs_to_edit[entry_name1][0]] + entry_seq1[dict_of_contigs_to_edit[entry_name1][1]:]
                        entry_seq1 = temp_seq1
                    #---remove contigs with number of nucleotides less than 'short_contig_cutoff' variable
                    if len(entry_seq1) > short_contig_cutoff:
                        temp_line1 = '>' + str(entry_name1) + '\n'
                        temp_line2 = str(entry_seq1) + '\n'
                        g.write(temp_line1)
                        g.write(temp_line2)
            entry_name1 = line1.replace('>', '').replace('\n', '')
            entry_seq1 = ''
        else:
            entry_seq1 += line1.replace('\n', '')
    #---write out last contig sequence
    temp_line1 = '>' + str(entry_name1) + '\n'
    temp_line2 = str(entry_seq1) + '\n'
    g.write(temp_line1)
    g.write(temp_line2)

end = time.time()
print(end-start)

114.08654689788818
